# Flow Routing Lesson 3.

# 1. The PriorityFloodFlowRouter.
- Some landscape evolution models generate a large quantity of depressions over time and need to have an efficient algorithm to solve them and route flow, with reduced execution time.
    - This is particularly the case with landslides, or glacial retreats and advances.
- Faced to these large execution times with the DepressionFinderAndRouter, Benjamin Campforts developed the PriorityFloodFlowRouter with the CSDMS Team. This Component encapsulates the Richdem python package developed by Barnes. Richdem route flow based on the Priority Flood Flow algorithm detailed in [Barnes et al., 2014](https://dx.doi.org/10.1016/j.cageo.2013.01.009).
    - The component combine the functionalities of flow directing, accumulation and depression filling.
    - This algorithm works differently from the FlowDirectorSteepest and D8 combined with the DepressionFinderAndRouter:
```    
Use a different scanning, starting from the open boundary of the grid.
Not stricto sensu steepest descent.

Breach algorithm to handle depressions.
Use the notion of depression outlets. If there's no depression, this depression outler is identic to the catchment outlet.
Add a layer called depression_free_elevations, which fills the depressions of the topography

- we create a total order priority queue called To_do, with initially no elements. We will add nodes into To_do ordered by elevation. This means that we will handle the nodes in To_do by popping the node with lowest elevation first.
- we start from the nodes of the open boundary of the grid:
    - we add them into To_do.
    - we set their receivers as themselves.
    - we set their catchment outlets and depression outlets as themselves too.
    - we consider them as Done.
- Then, we execute this while the To_do queue has nodes:
    - we popped the node R with lowest elevation from the To_do queue.
    - For all the neighbors of this node R, which have not been Done:
        - we set them as donors of this node R (R then become their receiver).
        - we set their catchment outlet and depression outlet to the cathment outlet and depression outlet of the node R.
        - we set their depression_free_elevation:
            - if their elevation is below the depression outlet, set to the elevation of the depression outlet.
            - otherwise 
        - we add them to the To_do queue.
    - we set the node R as Done.
 ```   
 (Reformulation by myself, based on Barnes et al., 2014)


- Let's try the PriorityFloodFlowRouter on the grid of Lesson 2, using the FlowDirectorD8:
    - We avoid to close the northern boundary, the component may not consider closed boundaries very well.

In [ ]:
# Import packages necessary for this lesson
import numpy as np # Array handling.
import matplotlib as mpl
import matplotlib.pyplot as plt # Graphic display

from landlab import RasterModelGrid # Grid management.

# Depression and flow handling
from landlab.components import (
    FlowDirectorSteepest,
    FlowDirectorD8,
    FlowDirectorMFD,
    FlowDirectorDINF,
    FlowAccumulator,
    DepressionFinderAndRouter,
    PriorityFloodFlowRouter,
    SpaceLargeScaleEroder,
)

# Flood status.
from landlab.components.depression_finder.floodstatus import FloodStatus

# import landlab plotting functionality.
from landlab import imshowhs_grid, imshow_grid
from landlab.plot.drainage_plot import drainage_plot
from landlab.plot import graph

In [ ]:
# Grid parameters.
grid_param = {
        "shape": (10, 10),
        "xy_spacing": (10, 10),
        "xy_axis_units": "m",
    }
# Creation of the grid.
g = RasterModelGrid(**grid_param)

# Close the northern boundary
g.set_closed_boundaries_at_grid_edges(
            right_is_closed=False,
            top_is_closed=False,
            left_is_closed=False,
            bottom_is_closed=False,
        )

# Create field topographic elevation at nodes with value 0 (will be modified later)
z = g.add_zeros("topographic__elevation", at="node", clobber=True)

# set constant random seed for "stable" random pull
random_generator = np.random.Generator(np.random.PCG64(seed=500))
random_topography = 10.0  # range in elevation (from 0)
z = g.at_node["topographic__elevation"] = (
    random_topography * random_generator.random(g.number_of_nodes) + 1.0
)
z[0] = 0.0

# Select the depression nodes and set their new lowered random topography.
depression_nodes = [43, 44, 45, 53, 54, 55, 63, 64, 65, 66, 74, 75]
z[depression_nodes] = random_generator.random(len(depression_nodes))

In [ ]:
# Use of the PriorityFloodFlowRouter.
flow_metric = "D8"
fa_PF = PriorityFloodFlowRouter(
    g,
    surface="topographic__elevation",
    flow_metric=flow_metric,
    suppress_out=False,
    depression_handler="fill",
    accumulate_flow=True,
)

fa_PF.run_one_step()

# Display of the donor/receiver relationships and drainage network.
drainage_plot(g, surf_cmap="terrain")
plt.show()
# Display of river discharge.
imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )

- The component provides a result, which may not always respect the Steepest Descent algorithm, since it starts from the lowest nodes at the boundary and don't scan the nodes as systematically as the Steepest algorithm. However, the Priority Flood Flow algorithm provide plausible river network that can be used to investigate erosion/sedimentation processes in a landscape evolution model.
- The component also fills depressions and provide a new field to the grid called depression_free_elevation:

In [ ]:
# Display the depression_free_elevation.
imshow_grid(
        g,
        "depression_free_elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Depression free elevation",
        show_elements=True,
    )

## 2. One-to-Multiple flow Dinf.
- Let's now combine the PriorityFloodFlowRouter with the one-to-multiple flow Dinf ([Tarboton, 1997](https://doi.org/10.1029/96WR03137)):

In [ ]:
# Use of the PriorityFloodFlowRouter.
flow_metric = "Dinf"
fa_PF = PriorityFloodFlowRouter(
    g,
    surface="topographic__elevation",
    flow_metric=flow_metric,
    suppress_out=False,
    depression_handler="fill",
    accumulate_flow=True,
)

fa_PF.run_one_step()

# Display of the donor/receiver relationships and drainage network.
drainage_plot(g, surf_cmap="terrain")
plt.show()

- Dinf allows to split routing in two adjacent flows, with proportions vary according the respective slope. 
    - Because of this rule, we see that not all nodes have multiple flow.
    - Here multiple flow doesn't change much the river network.
    - But it can allow to make the distribution of sediment more realistic.

<br>
Author: Sebastien Lenard<br>
Date: 2023, May.

### Click here to learn about <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>